In [3]:
import json, requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import feedparser
# from markdownify import markdownify

In [64]:
nature_url = "https://www.nature.com"
subject_url = nature_url + "/search?article_type=research, reviews, protocols&subject={}&date_range=last_7_days&order=date_desc"

In [65]:
subjects = ["Physics", "Chemistry", "Engineering", "Materials science", 
    "Energy science and technology", "Mathematics and computing", "Astronomy and planetary science", 
    "Business and industry", "Energy and society", "Geography", "Science", "technology and society", 
    "Social policy", "Politics and international relations","Computational biology and bioinformatics", 
    "Chemical biology, Biotechnology", "Information systems and information technology", "Computer science", 
    "Machine learning", "Image processing", "Signal processing"]

subjects = [i.lower().replace(" ","-") for i in subjects]

In [42]:
date_str = "20 March 2023"
date_obj = datetime.strptime(date_str, '%d %B %Y')
date_obj

datetime.datetime(2023, 3, 20, 0, 0)

In [44]:
r = requests.request("GET", subject_url.format("biochemistry"))
soup = BeautifulSoup(r.content, "html.parser")

In [53]:
soup.find("a",{"class":"c-card__link u-link-inherit"})["href"]

'/articles/s41467-023-37281-7'

In [60]:
url = nature_url+soup.find("a",{"class":"c-card__link u-link-inherit"})["href"]
r = requests.request("GET", url)
soup = BeautifulSoup(r.content, "html.parser")

In [63]:
soup.find("div", {"class": "c-article-body main-content"})

In [75]:
def news_content(url: str) -> str:
    
    r = requests.request("GET", url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    if soup.find("div", {"class": "c-article-body main-content"}):
        
        field = ','.join([i.lower() for i in soup.find("ul",{"class":"c-article-subjects u-clearfix u-hide-print"}).text.strip().split("\n\n\n")])
        
        html_content = soup.find("div",{"class":"c-article-body main-content"})
        soup.find("figcaption").decompose()
        
        content = html_content.text
        img_link = "https:" + soup.find("figure",{"class":"figure"}).img["src"]
        
        return field, content, img_link
        
    return None

In [76]:
def one_week_articles() -> list:

    articles = list()
    
    nature_rss = feedparser.parse("https://www.nature.com/nature.rss")
    
    for item in nature_rss["entries"]:
        
        try:
            
            article_link = item["id"]
        
            article_date = datetime.strptime(item["updated"], '%Y-%m-%d') 

            if (datetime.today() - timedelta(weeks=1)) < article_date < datetime.today():

                _data = news_content(article_link)
                field, _content, img_link = _data

                if _data is not None:

                    article = {}

                    article["title"] = item["title"]
                    article["summary"] = item["summary"]

                    article["field"] = field
                    article["content"] = _content
                    article["img_link"] = img_link

                    articles.append(article)
        except Exception as Error:
            
            print(f"Cant not scarpe { article_link } due to - {Error}")

    return articles

In [77]:
data = one_week_articles()

Cant not scarpe https://www.nature.com/articles/d41586-023-00844-1 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00845-0 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00843-2 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00708-8 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/s41586-023-05950-8 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/s41586-023-05958-0 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00799-3 due to - cannot unpack non-iterable NoneType object
Cant not scarpe https://www.nature.com/articles/d41586-023-00853-0 due to - 'NoneType' object has no attribute 'text'
Cant not scarpe https://www.nature.com/articles/d

In [79]:
len(data)

21

In [80]:
data

[{'title': 'How not to chatter like a toddler when giving a scientific presentation',
  'summary': '<p>Nature, Published online: 21 March 2023; <a href="https://www.nature.com/articles/d41586-023-00832-5">doi:10.1038/d41586-023-00832-5</a></p>Are we hardwired to overstuff presentations with details? Four simple steps can overcome this tendency, says David Rubenson.',
  'field': 'careers,conferences and meetings,communication',
  'content': '\n\n\n\n\n\n\nI have a new mentor: my two-and-a-half-year-old grandson. Like many toddlers, he rambles continuously, with little concern as to whether anyone understands him. It struck me that, if a toddler uses language in this way, it’s no surprise that scientific presentations — which I focus on in my job as a presentation coach — can themselves be overly detailed and undecipherable. A toddler’s loquaciousness creeps into many scientific talks — perhaps it’s an instinct we don’t completely outgrow.This might explain why making scientists aware of

In [8]:
r = requests.get("https://www.nature.com/articles/d41586-023-00795-7")
soup = BeautifulSoup(r.content, "html.parser")

In [9]:
soup.find("ul",{"class":"c-article-subjects u-clearfix u-hide-print"}).text.strip().split("\n\n\n")

['Careers', 'Physics', 'Nanoscience and technology']

In [42]:
soup.find("figure",{"class":"figure"}).img["src"]

'//media.nature.com/w767/magazine-assets/d41586-023-00795-7/d41586-023-00795-7_24630594.jpg'

In [26]:
_test_ls = [
    "https://www.nature.com/articles/d41586-023-00831-6",
    "https://www.nature.com/articles/d41586-023-00818-3",
    "https://www.nature.com/articles/d41586-023-00816-5"
]

In [100]:
soup.find("img",{"class":"figure__image"})["src"]

'//media.nature.com/lw767/magazine-assets/d41586-023-00831-6/d41586-023-00831-6_24636118.jpg'

In [28]:
lorem = """Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."""

articles = list()
for link in _test_ls:
    _data = article_content(link)
            
    if _data is not None:

        article = {}

        article["title"] = "Sample Title"
        article["summary"] = lorem

        _data = article_content(link)

        field, _content, img_link = _data

        article["field"] = field
        article["content"] = _content
        article["img_link"] = img_link
        
        articles.append(article)

In [29]:
from jinja2 import Environment, FileSystemLoader

In [30]:
articles

[{'title': 'Sample Title',
  'summary': "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum.",
  'field': 'Publishing',
  'content': '\nLast October, the pioneering life-sciences journal eLife introduced bold changes to its editorial practice — which some researchers applauded as reimagining the purpose of a scientific journal. From 31 January this year, eLife said, it would publish every paper it sent out for peer review: authors would never again receive a rejection afte

In [53]:
results_filename = "sample.html"
results_template = environment.get_template("index.html")
context = {
    "datetime" : tuple( ((datetime.today() - timedelta(weeks=1)).strftime("%d/%m/%Y"), datetime.today().strftime("%d/%m/%Y")))
    "data": data,
}
with open(results_filename, mode="w", encoding="utf-8") as results:
    results.write(results_template.render(context))
    print(f"... wrote {results_filename}")

... wrote sample.html


In [8]:
datetime.today().strftime("%d/%m/%Y")

'14/03/2023'